In [1]:
# 설치 안된 경우에는 
# !pip install cx_Oracle
import cx_Oracle
con = cx_Oracle.connect("bigdb/bigdb1234!@#$@115.21.253.187/xe")  # username/password@host/SID

In [3]:
cur = con.cursor()
cur.execute('SELECT * FROM item')
rows = cur.fetchall()
for row in rows:
    print(row)

('M0001', 'SPECIAL_SET', '온라인_전용상품', 'COMBO', 24000)
('M0002', 'PASTA', '파스타', 'FOOD', 12000)
('M0003', 'PIZZA', '피자', 'FOOD', 17000)
('M0004', 'SEA_FOOD', '씨푸드', 'FOOD', 25000)
('M0005', 'STEAK', '스테이크', 'FOOD', 35000)
('M0006', 'SALAD_BAR', '샐러드바', 'COMBO', 25000)
('M0007', 'SALAD', '샐러드', 'FOOD', 15000)
('M0008', 'SANDWICH', '샌드위치', 'FOOD', 10000)
('M0009', 'WINE', '와인', 'BEVERAGE', 8000)
('M0010', 'JUICE', '주스', 'BEVERAGE', 6000)


In [267]:
# pandas에서 sql 실행
import pandas as pd
item = pd.read_sql('SELECT * FROM item', con)
order_info = pd.read_sql('SELECT * FROM order_info', con)
reservation = pd.read_sql('SELECT * FROM reservation', con)
customer = pd.read_sql('SELECT * FROM customer', con)
address = pd.read_sql('SELECT * FROM address', con)

E:\kdigital_a\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
E:\kdigital_a\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
E:\kdigital_a\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
E:\kdigital_a\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 

In [2]:
import pickle
import gzip
data = {
    'item':item,
    'reservation':reservation,
    'order_info':order_info,
    'customer':customer,
    'address':address
}
# save and compress.
with gzip.open('restrant.pickle', 'wb') as f:
    pickle.dump(data, f)

NameError: name 'item' is not defined

In [55]:
import pickle, gzip
with gzip.open('restrant.pickle', 'rb') as f:
    data = pickle.load(f)
data

{'item':   ITEM_ID PRODUCT_NAME PRODUCT_DESC CATEGORY_ID  PRICE
 0   M0001  SPECIAL_SET     온라인_전용상품       COMBO  24000
 1   M0002        PASTA          파스타        FOOD  12000
 2   M0003        PIZZA           피자        FOOD  17000
 3   M0004     SEA_FOOD          씨푸드        FOOD  25000
 4   M0005        STEAK         스테이크        FOOD  35000
 5   M0006    SALAD_BAR         샐러드바       COMBO  25000
 6   M0007        SALAD          샐러드        FOOD  15000
 7   M0008     SANDWICH         샌드위치        FOOD  10000
 8   M0009         WINE           와인    BEVERAGE   8000
 9   M0010        JUICE           주스    BEVERAGE   6000,
 'reservation':       RESERV_NO RESERV_DATE RESERV_TIME CUSTOMER_ID BRANCH  VISITOR_CNT CANCEL
 0    2017082701    20170827        1800    W1340914     마포            2      N
 1    2017082602    20170826        1200     W341658     동작            5      N
 2    2017082601    20170826        1800    W1328993     강북            4      N
 3    2017082501    20170825        1800

In [56]:
item = data.get('item')
order_info = data.get('order_info')
reservation = data.get('reservation')
customer = data.get('customer')
address = data.get('address')

In [95]:
# 매출분석
import numpy as np
re = order_info['SALES'].agg({'총주문건': len, '총매출합계': sum, '평균매출': np.mean, '최고매출' :max, '최저매출':min}).astype(int)
re

총주문건          391
총매출합계    24957000
평균매출        63828
최고매출       552000
최저매출         6000
Name: SALES, dtype: int32

In [105]:
pd.DataFrame([order_info['SALES'].agg({'총주문건': len, '총매출합계': sum, '평균매출': np.mean, '최고매출' :max, '최저매출':min}).astype(int)])

,총주문건,총매출합계,평균매출,최고매출,최저매출
SALES,391,24957000,63828,552000,6000


In [106]:
# 3. 매출 비교분석
item, reservation,order_info,customer,address

(  ITEM_ID PRODUCT_NAME PRODUCT_DESC CATEGORY_ID  PRICE
 0   M0001  SPECIAL_SET     온라인_전용상품       COMBO  24000
 1   M0002        PASTA          파스타        FOOD  12000
 2   M0003        PIZZA           피자        FOOD  17000
 3   M0004     SEA_FOOD          씨푸드        FOOD  25000
 4   M0005        STEAK         스테이크        FOOD  35000
 5   M0006    SALAD_BAR         샐러드바       COMBO  25000
 6   M0007        SALAD          샐러드        FOOD  15000
 7   M0008     SANDWICH         샌드위치        FOOD  10000
 8   M0009         WINE           와인    BEVERAGE   8000
 9   M0010        JUICE           주스    BEVERAGE   6000,
       RESERV_NO RESERV_DATE RESERV_TIME CUSTOMER_ID BRANCH  VISITOR_CNT CANCEL
 0    2017082701    20170827        1800    W1340914     마포            2      N
 1    2017082602    20170826        1200     W341658     동작            5      N
 2    2017082601    20170826        1800    W1328993     강북            4      N
 3    2017082501    20170825        1800    W1340463     용산    

In [135]:
order_info['SALES'].sum()
item.head()
order_info[order_info['ITEM_ID'] == 'M0001']['SALES'].agg([len, sum])
re1 = pd.DataFrame([order_info['SALES'].agg({'총주문건수':len, '총매출합계':sum})])
re2 = pd.DataFrame([order_info[order_info['ITEM_ID'] == 'M0001']['SALES'].agg({'온라인전용상품주문건수':len, '온라인전용상품매출합계':sum})])
report2 = pd.concat([re1, re2], axis=1)
report2

,총주문건수,총매출합계,온라인전용상품주문건수,온라인전용상품매출합계
SALES,391,24957000,59,5808000


In [148]:
report2['주문건수비율'] = (report2['온라인전용상품주문건수']/report2['총주문건수']*100).round(2)
report2['매출합계비율'] = (report2['온라인전용상품매출합계']/report2['총매출합계']*100).round(2)
report2

,총주문건수,총매출합계,온라인전용상품주문건수,온라인전용상품매출합계,주문건수비율,매출합계비율
SALES,391,24957000,59,5808000,15.09,23.27


In [155]:
# 4. 상품별 매출분석
pd.merge(item, order_info, on='ITEM_ID').groupby(['ITEM_ID','PRODUCT_NAME'])['SALES'].sum()

ITEM_ID  PRODUCT_NAME
M0001    SPECIAL_SET     5808000
M0002    PASTA           1968000
M0003    PIZZA           1666000
M0004    SEA_FOOD        1625000
M0005    STEAK           9380000
M0006    SALAD_BAR       2075000
M0007    SALAD            525000
M0008    SANDWICH         610000
M0009    WINE             856000
M0010    JUICE            444000
Name: SALES, dtype: int64

In [166]:
report3 = pd.DataFrame(pd.merge(item, order_info, on='ITEM_ID').groupby(['ITEM_ID','PRODUCT_NAME'])['SALES'].sum())
report3 = report3.rename(columns={'SALES':'매출합계'})
report3.sort_values('매출합계', ascending=False)

,,매출합계
ITEM_ID,PRODUCT_NAME,
M0005,STEAK,9380000
M0001,SPECIAL_SET,5808000
M0006,SALAD_BAR,2075000
M0002,PASTA,1968000
M0003,PIZZA,1666000
M0004,SEA_FOOD,1625000
M0009,WINE,856000
M0008,SANDWICH,610000
M0007,SALAD,525000


In [214]:
# 5. 월별 상품 매출 분석(시계열 분석)
item, reservation,order_info,customer,address
order_info['RESERV_NO'] = order_info['RESERV_NO'].str[:6]
temp = pd.merge(order_info, item, on='ITEM_ID', how='left')
# temp['SALES'].fillna(0).astype(int)
temp = pd.pivot_table(temp, index='RESERV_NO', columns='PRODUCT_NAME', values='SALES', aggfunc=sum)
temp.fillna(0).astype(int)

PRODUCT_NAME,JUICE,PASTA,PIZZA,SALAD,SALAD_BAR,SANDWICH,SEA_FOOD,SPECIAL_SET,STEAK,WINE
RESERV_NO,,,,,,,,,,
201706,12000,24000,0,30000,0,10000,0,144000,385000,8000
201707,36000,228000,102000,0,200000,70000,175000,408000,525000,0
201708,42000,108000,136000,15000,175000,90000,225000,336000,455000,40000
201709,48000,120000,238000,90000,200000,30000,300000,264000,280000,16000
201710,102000,336000,391000,135000,425000,40000,200000,552000,1120000,32000
201711,174000,492000,544000,105000,400000,150000,425000,888000,1715000,304000
201712,30000,660000,255000,150000,675000,220000,300000,3216000,4900000,456000


In [17]:
# 6. 월별 총 매출, 전용상품 매출 비교분석
order_info.head()
order_info['RESERV_NO'] = order_info['RESERV_NO'].str[:6]
re6_1 = order_info.groupby('RESERV_NO')['SALES'].sum()
re6_2 = order_info[order_info['ITEM_ID']=='M0001'].groupby('RESERV_NO')['SALES'].sum()
report6 = pd.concat([re6_1, re6_2], axis=1)
report6 = pd.DataFrame(report6)
report6.columns = ['매출합계', 'SPECIAL_SET']
report6

,매출합계,SPECIAL_SET
RESERV_NO,,
201706,613000,144000
201707,1744000,408000
201708,1622000,336000
201709,1586000,264000
201710,3333000,552000
201711,5197000,888000
201712,10862000,3216000


In [48]:
# 7. 매출 기여율
import pandas as pd
import numpy as np
report7 = report6.copy()
report7['년월'] = report7.index
report7['ETC'] = report6['매출합계']-report6['SPECIAL_SET']
# report6['매출합계']-report6['SPECIAL_SET']
report7['매출기여율'] = round(report6['SPECIAL_SET'] / report6['매출합계'] * 100 ,1)
report7 = report7[['년월', 'ETC', 'SPECIAL_SET', '매출기여율']]
report7.reset_index(drop=True, inplace=True)
report7.index = np.arange(1, len(report7)+1)
report7

,년월,ETC,SPECIAL_SET,매출기여율
1,201706,469000,144000,23.5
2,201707,1336000,408000,23.4
3,201708,1286000,336000,20.7
4,201709,1322000,264000,16.6
5,201710,2781000,552000,16.6
6,201711,4309000,888000,17.1
7,201712,7646000,3216000,29.6
